In [11]:
import re
import requests
import math
from lxml import html
from lxml.html import tostring
from collections import Counter
from string import punctuation as enPunc
from zhon.hanzi import punctuation as zhPunc
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from bs4  import BeautifulSoup
from LAC import LAC
from opencc import OpenCC

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 資料預處理

In [12]:
## 振華數據
search_word = '振华数据'
google_search_rs = pd.read_excel('ner_sample.xlsx').url.to_list()

### Get Link Text

In [14]:
html_text = []
getLink = []
for idx, link in enumerate(google_search_rs):
    getLink.append((idx+1, link))
    print(idx+1, link)
    ## 用get下載普通網頁 (把每個連結的資料下載下來，並自訂表頭headers)
    resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
    soup = BeautifulSoup(resp.content, 'html.parser')
    a_tag = soup.find_all('a')
    for a in a_tag:
        a.decompose()   
    html_text.append(soup.get_text())    

1 https://www.qcc.com/firm/6a477279b2ee73e05e11f607554cb084.html
2 https://www.enanyang.my/%E5%9B%BD%E9%99%85/%E6%8C%AF%E5%8D%8E%E6%95%B0%E6%8D%AE%E7%BD%91%E7%AB%99%E5%B7%B2%E4%B8%8B%E7%BA%BF-%E7%8E%8B%E9%9B%AA%E5%B3%B0%E9%BC%93%E5%90%B9%E2%80%9C%E6%B7%B7%E5%90%88%E6%88%98%E4%BA%89%E2%80%9D
3 https://tw.appledaily.com/politics/20200920/RA2DTEHKGNCNHHQFUEM7PSZX4M/
4 https://www.epochtimes.com/gb/20/9/15/n12404579.htm
5 https://www.epochtimes.com/b5/20/9/15/n12404579.htm
6 https://zh.wikipedia.org/zh-tw/%E6%B7%B1%E5%9C%B3%E6%8C%AF%E5%8D%8E%E6%95%B0%E6%8D%AE%E4%BF%A1%E6%81%AF%E6%8A%80%E6%9C%AF%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
7 https://www.bbc.com/zhongwen/trad/chinese-news-54160466
8 https://twitter.com/watchwci/status/1305282854562279425
9 https://cn.unionpeace.org/people/12934.html
10 https://www.abc.net.au/chinese/2020-09-14/chinese-data-leak-linked-to-military-names-australians/12661676
11 https://www.youtube.com/watch?v=XCJ68bNPlOA
12 https://gnews.org/zh-hant/356177/
13 https://

2021-05-24 14:09:22,917 - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


48 http://read01.com/zh-tw/2Kk0ODx.html
49 https://www.tiaotiaowen.com/articles/bsd8wy


### Setup stopwords
- 刪除英文字
- 繁轉簡
- 建立特定符號 ['\n', '\r', '\t', '\xa0'] 及中英標點 的list
- 匯入stop words檔案 (Biadu stop words)
- 建立刪除字符的dictionary

In [15]:
# Load stop words
f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
k = f.readlines()
f.close()
stopwords = [lines.strip('\n') for lines in k]
# for i in search_word:
#     stopwords.append(cc.convert(i))  ## 加入搜尋詞，避免影響關鍵字萃取結果

# 建構stopwords
trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
trim = trim + stopwords  # list of all stopwords
# trim_dict = {pat:'' for pat in trim}

# 繁轉簡
cc = OpenCC('tw2sp')
temp = [cc.convert(texts) for texts in html_text]

# 去除英文
final_text = [re.sub(r'[a-zA-Z]',"", texts) for texts in temp]

In [5]:
# def multi_repl(input_str, repl_dict):
#     for k in repl_dict:
#         input_str = input_str.replace(k, repl_dict[k])  ## 把stop words改為空白
#     return input_str

In [210]:
## 分詞前清除stopwords
# final_text = [multi_repl(texts,trim_dict) for texts in temp]
# final_text

### Word Segmentation & NER (using LAC model)

In [16]:
lac = LAC(mode='lac')  
lac.load_customization('custom_dict.txt')  # 自行增加字典

# input html_text which hasn't remove stopwords
ner_model = lac.run(final_text)  # [[words, labels], [words, labels],....]

In [61]:
## 分詞後整合
ner_pair = []   # 分詞結果
docs = pd.DataFrame(columns=['texts', 'doc'])  # dataframe 每篇文章為一列
for doc, (word, ner) in enumerate(ner_model):
    temp = []
    for num in range(len(word)):
        if word[num] not in trim: # remove stop words
            if word[num] != '' and ner[num] != '' and ner[num] != 'f' and ner[num] != 'nw' and ner[num] != 'm':
                temp.append(word[num])
                ner_pair.append((word[num],ner[num],doc+1))
    docs = docs.append({'texts': temp, 'doc': doc+1}, ignore_index = True)

## Sequence Tagging 
- model selected: LAC
- already finished (ref. 1.3)

In [ ]:
## 查看分詞、NER結果  -- lots of noise 
for idx, (word, ner) in enumerate(ner_pair):
    for num in range(len(word)):
        print((word[num],ner[num]))

### [replace] Remove stop words/ noises

####  remove noise: some noise NER and 標點 and 空值 and 數量

In [24]:
filtered_ner = []
cc = OpenCC('tw2sp')
for idx, (word, ner) in enumerate(ner_pair):
    for num in range(len(word)):
        if word[num] not in stopwords:
            word[num] = multi_repl(word[num],trim_dict)  # remove stop words
            if ner[num] != '' and ner[num] != 'w' and word[num] != '' and ner[num] != 'm': # remove some noise NER
                cv_word = cc.convert(word[num])  ## 使用繁轉簡
                print((cv_word,ner[num]))
                filtered_ner.append((cv_word,ner[num]))

('深圳', 'ORG')
('振华数据', 'ORG')
('信息技术有限公司', 'ORG')
('深圳', 'ORG')
('振华数据', 'ORG')
('信息技术有限公司', 'ORG')
('企业', 'n')
('主页', 'n')
('品牌', 'n')
('浏览量', 'n')
('访问量', 'n')
('即可', 'v')
('查看', 'v')
('访问', 'vn')
('记录', 'n')
('认证', 'v')
('自己', 'r')
('深圳', 'ORG')
('振华数据', 'ORG')
('信息技术有限公司', 'ORG')
('存续', 'v')
('高新技术', 'n')
('企业', 'n')
('科技型', 'a')
('中小企业', 'n')
('小微', 'a')
('信息', 'n')
('传输', 'vn')
('软件', 'n')
('信息技术', 'n')
('服务业', 'n')
('软件', 'n')
('信息技术', 'n')
('服务业', 'n')
('更多', 'a')
('信息', 'n')
('传输', 'vn')
('软件', 'n')
('信息技术', 'n')
('服务业', 'n')
('软件', 'n')
('信息技术', 'n')
('电话', 'n')
('企业', 'n')
('官网', 'n')
('地址', 'n')
('深圳市', 'LOC')
('南山区', 'LOC')
('南头街道', 'LOC')
('莲城社区', 'LOC')
('深南大道', 'LOC')
('南山软...', 'ORG')
('附近', 'f')
('企业', 'n')
('最新', 'a')
('年报', 'n')
('地址', 'n')
('简介', 'n')
('振华', 'ORG')
('聚焦', 'v')
('集成', 'v')
('境外', 's')
('数据', 'n')
('信息', 'n')
('国内', 's')
('机构', 'n')
('提供', 'v')
('服务', 'vn')
('包括', 'v')
('境外', 's')
('数据', 'n')
('采集', 'vn')
('数据', 'n')
('咨询', 'vn')
('数据', 'n')
('集成', '

('合伙', 'vn')
('事务', 'n')
('执行人', 'n')
('变更', 'vn')
('高级', 'a')
('管理人员', 'n')
('备案', 'v')
('董事', 'n')
('监事', 'n')
('经理', 'n')
('地址', 'n')
('变更', 'v')
('住所', 'n')
('地址', 'n')
('经营', 'vn')
('场所', 'n')
('驻', 'v')
('地址', 'n')
('变更', 'vn')
('深圳市', 'LOC')
('南山区', 'LOC')
('南山街道', 'LOC')
('南海大道', 'LOC')
('西桂庙路北阳光华艺大厦', 'LOC')
('6B', 'n')
('深圳市', 'LOC')
('南山区', 'LOC')
('南头街道', 'LOC')
('大汪山', 'LOC')
('社区', 'n')
('桃园东路', 'LOC')
('西海明珠大厦', 'LOC')
('从事', 'v')
('信息技术', 'n')
('电子产品', 'n')
('领域', 'n')
('内', 'f')
('技术', 'n')
('开发', 'vn')
('技术', 'n')
('咨询', 'vn')
('技术', 'n')
('服务', 'vn')
('信息', 'n')
('咨询', 'vn')
('不含', 'v')
('限制', 'vn')
('项目', 'n')
('互联网', 'n')
('数据', 'n')
('信息', 'n')
('分析', 'vn')
('大数据', 'p')
('领域', 'n')
('内', 'f')
('技术', 'n')
('开发', 'vn')
('人工智能', 'n')
('科技', 'n')
('开发', 'vn')
('以上', 'f')
('根据', 'p')
('法律', 'n')
('行政', 'n')
('法规', 'n')
('国务院', 'ORG')
('决定', 'v')
('规定', 'n')
('需要', 'v')
('审批', 'vn')
('依法', 'vd')
('取得', 'v')
('相关', 'vn')
('审批', 'vn')
('文档', 'n')
('后方', 'f')
('经营', 'v')
(

('王雪峰', 'PER')
('美国', 'LOC')
('科技', 'n')
('巨擘', 'n')
('国际商业机器公司', 'ORG')
('IBM', 'ORG')
('雇员', 'n')
('微信', 'nz')
('支持', 'v')
('通过', 'p')
('操纵', 'v')
('民意', 'n')
('心理战', 'n')
('进行', 'v')
('混合', 'vn')
('战争', 'n')
('hybrid', 'nz')
('war', 'nz')
('王雪峰', 'PER')
('IBM', 'ORG')
('工作', 'v')
('期间', 'f')
('带领', 'v')
('专家', 'n')
('团队', 'n')
('数据', 'n')
('中心', 'n')
('提供', 'v')
('技术', 'n')
('服务', 'vn')
('2015', 'TIME')
('年底', 'TIME')
('正式', 'ad')
('带领', 'v')
('团队', 'n')
('离开', 'v')
('公司', 'n')
('独立', 'ad')
('创业', 'v')
('据报道', 'd')
('振华数据', 'ORG')
('2018年', 'TIME')
('成立', 'v')
('据悉', 'd')
('中国振华电子集团', 'ORG')
('所有', 'r')
('后者', 'r')
('国营', 'a')
('中国电子信息产业集团', 'ORG')
('CETC', 'ORG')
('所有', 'r')
('CETC', 'ORG')
('军事', 'n')
('研究', 'vn')
('公司', 'n')
('公司', 'n')
('悉尼科技大学', 'ORG')
('2019年', 'TIME')
('之前', 'u')
('合作', 'vn')
('振华数据', 'ORG')
('执行长', 'n')
('王雪峰', 'PER')
('照片', 'n')
('这家', 'r')
('位于', 'v')
('深圳', 'LOC')
('公司', 'n')
('花', 'v')
('利用', 'v')
('人工智能', 'n')
('大数据', 'n')
('建构', 'v')
('庞大', 'a')
('数据库'

('中国共产党', 'ORG')
('振华', 'PER')
('对', 'p')
('各国', 'r')
('搜集', 'v')
('海外', 's')
('重要', 'a')
('人士', 'n')
('数据库', 'n')
('OverseasKey', 'nz')
('InformationDatabase', 'nz')
('OKIDB', 'nz')
('存有', 'v')
('多达', 'v')
('个人', 'n')
('数据', 'n')
('其中', 'f')
('约', 'd')
('文件', 'n')
('Internet', 'n')
('成功', 'ad')
('复原', 'v')
('数据', 'n')
('包含', 'v')
('美国人', 'n')
('为', 'v')
('最多', 'a')
('其次', 'r')
('澳洲人', 'nz')
('另外', 'c')
('也有', 'v')
('印度', 'n')
('英国人', 'nz')
('加拿大', 'LOC')
('以及其他', 'r')
('国家', 'n')
('公民', 'n')
('该', 'r')
('数据库', 'n')
('透过', 'v')
('爬梳推特', 'nz')
('脸书', 'n')
('领', 'v')
('英', 'LOC')
('Instagram', 'n')
('抖', 'v')
('音', 'n')
('来', 'v')
('搜罗', 'v')
('信息', 'n')
('并', 'c')
('详列', 'v')
('个人', 'n')
('出生', 'v')
('年月日', 'n')
('地址', 'n')
('婚姻', 'n')
('状况', 'n')
('照片', 'n')
('政治', 'n')
('活动', 'n')
('据信', 'v')
('中国', 'LOC')
('情治单位', 'n')
('与', 'p')
('国家', 'n')
('安全部', 'n')
('都会', 'v')
('使用', 'v')
('这些', 'r')
('信息', 'n')
('更多名单', 'n')
('请见', 'v')
('解密', 'v')
('振华', 'vn')
('数据', 'n')
('查查', 'v')
('是否', '

('michael', 'nz')
('ASPI', 'n')
('September', 'n')
('东', 'f')
('警惕', 'v')
('微信', 'nz')
('中共国际超限战大杀器', 'nz')
('\xa02019-08-22', 'nz')
('美国', 'LOC')
('智库', 'n')
('建议', 'v')
('政府', 'n')
('推倒', 'v')
('中共防火墙图\xa02020-05-12大陆公司搜集240万境外个人数据库被曝光\xa02020-09-14大陆', 'nw')
('融媒体', 'nz')
('瞄准', 'v')
('台青', 'n')
('中共统战新工具图\xa02020-09', 'nw')
('中共', 'f')
('搜集', 'v')
('外国人', 'n')
('数据', 'n')
('每人', 'r')
('都', 'd')
('编号', 'v')
('\xa0\xa02020-09-14澳统计局', 'nw')
('季度', 'n')
('消费', 'vn')
('价格', 'n')
('指数', 'n')
('上升', 'v')
('\xa0', 'n')
('2021-04-28', 'TIME')
('应印', 'vn')
('太', 'd')
('局势', 'n')
('澳斥', 'v')
('巨资', 'n')
('升级', 'v')
('北领地军事基地图\xa02021-04-28脸书删除澳议员凯利的备用账号图\xa02021-04-28悉尼北海滩污水一周内两次验出病毒碎片图\xa02021-04-28澳移民部长', 'nw')
('澳', 'LOC')
('仍可', 'd')
('出国', 'v')
('热门', 'a')
('新闻', 'n')
('大学', 'n')
('进入', 'v')
('世界', 'n')
('审查', 'vn')
('澳', 'LOC')
('外长', 'n')
('废除', 'v')
('更多', 'a')
('涉外', 'vn')
('2021年', 'TIME')
('超级月亮', 'n')
('今晚', 'TIME')
('高悬', 'v')
('澳洲', 'LOC')
('莫里森', 'PER')
('上帝', 'n')
('召唤', 'v')

...

('人工智能', 'n')
('科技', 'n')
('监控', 'vn')
('相关', 'vn')
('议题', 'n')
('美国维吉尼亚大学', 'ORG')
('传媒', 'n')
('教授', 'n')
('孔安怡', 'PER')
('AynneKokas', 'nz')
('告诉', 'v')
('自由亚洲', 'LOC')
('电台', 'n')
('这', 'r')
('类', 'q')
('数据', 'n')
('搜集', 'vn')
('使用', 'v')
('方面', 'n')
('训练军', 'n')
('商用', 'v')
('人工智能算法', 'nz')
('信息战', 'n')
('则', 'd')
('锁定', 'v')
('特定', 'a')
('人员', 'n')
('情报', 'n')
('工作', 'n')
('台', 'n')
('台湾', 'LOC')
('数据库', 'n')
('发现', 'v')
('大多', 'd')
('详细', 'ad')
('记录', 'v')
('个人', 'n')
('最新', 'a')
('法庭', 'n')
('记录', 'v')
('比如', 'v')
('走私', 'v')
('贪', 'v')
('腐', 'an')
('收', 'v')
('贿', 'n')
('各种', 'r')
('细节', 'n')
('鲍尔丁', 'PER')
('说', 'v')
('他们', 'r')
('其他', 'r')
('国家', 'n')
('发现', 'v')
('这个', 'r')
('特性', 'n')
('鲍尔丁', 'PER')
('我们', 'r')
('发现', 'v')
('他们', 'r')
('搜集', 'v')
('各国', 'n')
('组织', 'n')
('犯罪者', 'n')
('数据', 'n')
('像你', 'r')
('说', 'v')
('法庭', 'n')
('文档', 'n')
('我们', 'r')
('只能', 'v')
('猜测', 'v')
('他们', 'r')
('列出', 'v')
('可能', 'v')
('可以', 'v')
('影响', 'v')
('一个人', 'n')
('方方面面', 'n')
('除了', 'p')

('史美伦', 'PER')
('信报', 'n')
('朱镕基', 'PER')
('习近平', 'PER')
('文昌讲政经(ruplayers.com/tools/lqWEtr3Vl2KorGg/video.html', 'ORG')
('快必', 'd')
('谭得志文本狱', 'PER')
('国安处', 'n')
('李桂华', 'PER')
('迟', 'ad')
('拉', 'v')
('证失职', 'v')
('雷鼎鸣', 'PER')
('教', 'v')
('精林', 'n')
('郑月娥', 'PER')
('贱', 'a')
('招迫', 'v')
('香港', 'LOC')
('全民', 'n')
('检测', 'v')
('教育局', 'n')
('杨润雄', 'PER')
('河蟹', 'n')
('耶稣', 'PER')
('爱', 'v')
('中国', 'LOC')
('D100', 'nz')
('风波', 'n')
('里', 'f')
('茶杯', 'n')
('足本', 'd')
('重温', 'v')
('主持', 'n')
('郑家富', 'PER')
('梁家权', 'n')
('ruplayers.com/tools/sJSw1b2tbWKormQ/video.html', 'ORG')
('陈彦霖', 'PER')
('离世', 'v')
('爆', 'v')
('林郑月娥', 'PER')
('政府', 'n')
('邓炳强', 'PER')
('同袍', 'n')
('玩', 'v')
('拖延', 'v')
('调查', 'v')
('新西兰澳洲', 'LOC')
('搞', 'v')
('全民', 'n')
('检测', 'v')
('健康码', 'n')
('旅游', 'vn')
('气泡头', 'n')
('都', 'd')
('传媒', 'n')
('黎智英', 'PER')
('玩', 'v')
('雷声', 'n')
('大雨', 'a')
('点', 'q')
('维多利亚讲', 'nz')
('林忌', 'n')
('林浩基', 'PER')
('ruplayers.com/tools/q5Glr5q4nWNqsIE/video.html', 'ORG')
('香港', 'LOC')
('

('系统', 'n')
('有关联', 'v')
('广东省', 'LOC')
('深圳市', 'LOC')
('振华数据', 'ORG')
('公司', 'ORG')
('标榜', 'v')
('自己', 'r')
('利用', 'v')
('遍布', 'v')
('全球', 'n')
('收集', 'vn')
('创建', 'v')
('超过', 'v')
('海外', 's')
('重要', 'a')
('人士', 'n')
('数据库', 'n')
('台', 'n')
('取得', 'v')
('相关', 'vn')
('资料', 'n')
('台湾', 'LOC')
('名单', 'n')
('首次', 'd')
('曝光', 'v')
('包含', 'v')
('马英九', 'PER')
('宋楚瑜', 'PER')
('台湾', 'LOC')
('政要', 'n')
('科技业', 'n')
('老板', 'n')
('及其', 'c')
('家人', 'n')
('都', 'd')
('数据库', 'n')
('中国深圳', 'ORG')
('振华数据', 'ORG')
('信息公司', 'ORG')
('ZhenhuaData', 'nz')
('泄漏', 'v')
('数据', 'n')
('显示', 'v')
('这家', 'r')
('中国人民解放军', 'ORG')
('中共安全情报系统有关连的中国公司', 'ORG')
('创建', 'v')
('约240万', 'd')
('全球', 'n')
('影响力', 'n')
('人士', 'n')
('OKIDB', 'n')
('个人', 'n')
('资料库', 'n')
('振华数据', 'ORG')
('公司', 'n')
('网站', 'n')
('介绍', 'v')
('OKIDB', 'n')
('资料库', 'n')
('网站', 'n')
('无法', 'vd')
('登录', 'v')
('研究者', 'n')
('台', 'n')
('提供', 'v')
('台湾', 'LOC')
('名单', 'n')
('首次', 'd')
('曝光', 'v')
('马英九', 'PER')
('宋楚瑜', 'PER')
('吴钊燮', 'PER')
('上榜', 'v')
(

('我们', 'r')
('放出', 'v')
('报告', 'n')
('数据库', 'n')
('原因', 'n')
('更是', 'd')
('希望', 'v')
('大众', 'n')
('知道', 'v')
('中国', 'LOC')
('搜集', 'v')
('海外', 's')
('个人', 'n')
('数据', 'n')
('行为', 'n')
('正在', 'd')
('发生', 'v')
('开放', 'v')
('自由', 'a')
('民主', 'a')
('国家', 'n')
('个人', 'n')
('机构', 'n')
('应该', 'v')
('更加', 'd')
('意识到', 'v')
('中国共产党', 'ORG')
('国家安全', 'n')
('情报', 'n')
('单位', 'n')
('正在进行', 'v')
('不对称', 'a')
('信息战', 'n')
('海外', 's')
('人士', 'n')
('隐私权', 'n')
('侵犯', 'vn')
('深圳', 'LOC')
('数据', 'n')
('库里', 'PER')
('台湾', 'LOC')
('名单', 'n')
('都', 'd')
('什幺', 'r')
('主持人', 'n')
('泄露', 'v')
('深圳', 'ORG')
('振华数据', 'ORG')
('库', 'n')
('台湾', 'LOC')
('部分', 'n')
('台', 'n')
('独家', 'd')
('取得', 'v')
('共有', 'v')
('台籍', 'n')
('人士', 'n')
('个人', 'n')
('资料', 'n')
('现在', 'TIME')
('请', 'v')
('记者', 'n')
('陈品洁', 'PER')
('详细', 'ad')
('介绍', 'v')
('这个', 'r')
('数据库', 'n')
('里面', 'f')
('什幺', 'r')
('栏目', 'n')
('记者', 'n')
('这个', 'r')
('台湾', 'LOC')
('数据库', 'n')
('报告', 'vn')
('作者', 'n')
('提供', 'v')
('英文', 'nz')
('方式', 'n')
('呈现', 'v')

('Johnston', 'ORG')
('振华数据', 'ORG')
('库', 'n')
('监控', 'vn')
('名单', 'n')
('称', 'v')
('都', 'd')
('感到', 'v')
('意外', 'n')
('访者', 'n')
('提供', 'v')
('目前', 'TIME')
('解密', 'v')
('资料库', 'n')
('发现', 'v')
('加拿大', 'LOC')
('名字', 'n')
('超过', 'v')
('关于', 'p')
('加拿大', 'LOC')
('条目', 'n')
('加拿大', 'LOC')
('政治', 'n')
('法律', 'n')
('学术', 'n')
('商业', 'n')
('犯罪者', 'n')
('人物', 'n')
('讯息', 'n')
('归纳', 'v')
('三等级', 'n')
('分配', 'v')
('似乎', 'd')
('直接', 'a')
('影响力', 'n')
('例如', 'c')
('国会', 'n')
('议员', 'n')
('高级', 'a')
('公务员', 'n')
('学术', 'n')
('研究者', 'n')
('刚', 'd')
('当选', 'v')
('保守党党', 'ORG')
('领', 'v')
('奥图尔', 'PER')
('ErinO', 'n')
('Toole', 'nz')
('在其内', 'd')
('分配', 'v')
('通常', 'd')
('当权者', 'n')
('亲戚', 'n')
('例如', 'c')
('总理', 'n')
('特鲁多', 'PER')
('女儿', 'n')
('国会议员', 'n')
('费凯迪', 'PER')
('儿子', 'n')
('等级', 'n')
('定罪者', 'n')
('主要', 'ad')
('经济', 'n')
('犯罪', 'vn')
('人物', 'n')
('玛格丽特·麦凯格', 'PER')
('约翰斯顿', 'PER')
('发现', 'v')
('加拿大', 'LOC')
('中国', 'LOC')
('企业', 'n')
('脱离', 'v')
('监控', 'v')
('范围', 'n')
('例如', 'c')
('加拿大核

#### choose specific NER: ORG, PER, TITLE, vd(動副詞), vn(動名詞), ad(形副詞), an(名形詞)
1. 繁轉簡
2. remove stop words (baidu)
3. 刪除文字為空者

- 原本有加入LOC,但國家名稱在詞頻方法中有非常多為common，因此移除LOC


In [9]:
## filtered_ner1, ner_lst
cc = OpenCC('tw2sp')
ner_lst = []
filtered_ner1 = []
for idx, (word, ner) in enumerate(ner_pair):
    temp = []
    for num in range(len(word)):
        if word[num] not in stopwords:
            word[num] = multi_repl(word[num],trim_dict)  # remove stop words
            if ner[num] == 'ORG' or ner[num] == 'PER' or ner[num] == 'TITLE': #or ner[num] == 'vd' or ner[num] == 'vn' or ner[num] == 'ad' or ner[num] == 'an': # remove some noise NER
                if word[num] != '':
                    cv_word = cc.convert(word[num])  ## 使用繁轉簡
#                     print((cv_word,ner[num]))  
                    filtered_ner1.append((cv_word,ner[num])) 
                    temp.append(cv_word)
    ner_lst.append(temp)

each_cnt = [Counter(row) for row in ner_lst]

## Setup Target Page

### 組合ner - org_title_per, org_per
- 將組合詞替代原先的分詞
- 包含ner組合後 & group 的 word list:  [('深圳', 'ORG', 1),('振华数据', 'ORG', 1),...

In [26]:
for idx, (word, ner, cat) in enumerate(ner_pair):
    if (idx + 1) == len(ner_pair) or (idx + 2) == len(ner_pair):
        break
    else:  # 直接將組合詞加入原先分詞好的list (ner_pair)
        n1_ner = ner_pair[idx+1]
        if cat == n1_ner[2]:
            n2_ner = ner_pair[idx+2]
            if ner == 'ORG' and n1_ner[1] == 'PER':
                new_wd = (word+n1_ner[0],'ORG+PER',cat)
                del ner_pair[idx:idx+2]
                ner_pair.insert(idx,new_wd) 
            elif ner == 'ORG' and n1_ner[1] == 'TITLE' and n2_ner[1] == 'PER':
                new_wd = (word+n1_ner[0]+n2_ner[0],'ORG+TITLE+PER',cat)
                del ner_pair[idx:idx+3]
                ner_pair.insert(idx,new_wd)

### 詞頻排序 - (直觀)只有ORG+TITLE+PER, ORG+PER

In [64]:
# df_all_pair[df_all_pair['cat'] == 2][40:80]
df_all_pair[:10]

,words,ner,cat
0,深圳,LOC,1
1,振华数据,ORG,1
2,信息技术有限公司,ORG,1
3,浏览量,n,1
4,近,ad,1
5,访问量,n,1
6,即可,v,1
7,查看,v,1
8,访问,vn,1
9,记录,n,1


In [27]:
df_all_pair = pd.DataFrame(ner_pair,columns=['words','ner','cat'])
# dataframe ORG+PER, ORG+TITLE+PER
df1 = df_all_pair[df_all_pair['ner'] == 'ORG+PER']
df2 = df_all_pair[df_all_pair['ner'] == 'ORG+TITLE+PER']
df_ppl_pair = pd.concat((df1,df2),ignore_index=True)
df_ppl_pair[:10]

,words,ner,cat
0,澳大利亚弗林德斯大学罗杰斯,ORG+PER,32
1,全美第五大投资银行贝尔斯登,ORG+PER,40
2,军事科学院国防科技创新研究院栗琳,ORG+PER,41
3,中国科学技术信息研究所刘耀,ORG+PER,41
4,中国社会科学院梁俊兰,ORG+PER,41
5,信息技术公司王雪峰,ORG+PER,41
6,国安处李桂华,ORG+PER,47
7,282壹传媒黎智英,ORG+PER,47
8,壹传媒创办人黎智英,ORG+TITLE+PER,3
9,振华首席执行官王雪峰,ORG+TITLE+PER,3


In [29]:
cnt = Counter(df_ppl_pair.cat).most_common()
for i in cnt:
    print('doc:%s, freq:%s'%(i[0],i[1]))

doc:41, freq:9
doc:49, freq:4
doc:23, freq:3
doc:24, freq:3
doc:45, freq:3
doc:32, freq:2
doc:47, freq:2
doc:3, freq:2
doc:10, freq:2
doc:16, freq:2
doc:28, freq:2
doc:44, freq:2
doc:40, freq:1
doc:7, freq:1
doc:9, freq:1
doc:13, freq:1
doc:15, freq:1
doc:38, freq:1


In [30]:
link = pd.DataFrame(getLink, columns=['doc','link'])
link['cnt'] = 0
for i in cnt:
    for idx in link.doc:
        if i[0] == idx:
            link['cnt'][idx-1] = i[1]

<ipython-input-30-868f442a03c2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['cnt'][idx-1] = i[1]


In [31]:
link.sort_values(by=['cnt'],ascending=False, ignore_index=True)

,doc,link,cnt
0,41,http://www.bast.net.cn/art/2019/12/25/art_2334...,9
1,49,https://www.tiaotiaowen.com/articles/bsd8wy,4
2,24,https://www.rfa.org/mandarin/yataibaodao/junsh...,3
3,23,https://www.fountmedia.io/article/75492,3
4,45,https://www.taiwanjustice.net/2020/09/15/%E8%8...,3
5,10,https://www.abc.net.au/chinese/2020-09-14/chin...,2
6,28,https://www.rti.org.tw/news/view/id/2079508,2
7,16,https://newsin.us/%E6%8C%AF%E5%8D%8E%E6%95%B0%...,2
8,44,https://www.mingjingnews.com/article/195941-01,2
9,32,https://www.dw.com/zh/%E5%B0%8F%E5%BF%83%E6%95...,2


### TF-IDF萃取十大關鍵字、標註NER、

In [71]:
# dataframe all ner
df_all_pair = pd.DataFrame(ner_pair,columns=['words','ner','cat'])
df_all_pair.ner.unique()

array(['LOC', 'ORG', 'n', 'ad', 'v', 'vn', 'a', 's', 'nz', 'p', 'vd',
       'TITLE', 'q', 'd', 'r', 'an', 'w', 'TIME', 'PER', 'u', 'c', 'xc'],
      dtype=object)

In [68]:
doc_term_freq = []  # 每篇文章每個詞的頻率
doc_terms = []  # 每篇文章的詞 (normalized)
for i in range(len(final_text)):
    tempDF = df_all_pair[df_all_pair['cat']==i+1]
    terms = tempDF['words'].unique()
    doc_terms.append(terms)
    cnt = Counter(tempDF.words)
    doc_term_freq.append(cnt)
# doc_term_freq

In [69]:
# 计算tf
def tf(word, count):
    return count[word] / sum(count.values())

#计算count_list有多少个文件包含word
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)

# 计算idf
def idf(word, count_list):
    return math.log(len(count_list) / (n_containing(word, count_list)))  

# 计算tf-idf
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

In [70]:
all_sorted = {x:[] for x in ['words', 'scores', 'doc']}  ## 存放每篇文章所萃取出的關鍵字
for i, count in enumerate(doc_term_freq):
    print("Top words in document %d"%(i + 1))
    scores = {word: tfidf(word, count, doc_term_freq) for word in count}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)    #type=list
    for word, score in sorted_words[:10]:
        all_sorted['words'].append(word)
        all_sorted['scores'].append(score)
        all_sorted['doc'].append(i+1)
        print("    Word: %s, TF-IDF: %s"%(word, round(score, 5)))
    print('\n')

Top words in document 1
    Word: 变更, TF-IDF: 0.06221
    Word: 咨询, TF-IDF: 0.05832
    Word: 历史, TF-IDF: 0.04925
    Word: 需, TF-IDF: 0.04186
    Word: 审批, TF-IDF: 0.03888
    Word: 开发, TF-IDF: 0.0381
    Word: 出资, TF-IDF: 0.0311
    Word: 添加, TF-IDF: 0.03069
    Word: 电子产品, TF-IDF: 0.02722
    Word: 南头街道, TF-IDF: 0.02333


Top words in document 2
    Word: 下线, TF-IDF: 0.0638
    Word: 鼓吹, TF-IDF: 0.04253
    Word: 可用于, TF-IDF: 0.04253
    Word: 带领, TF-IDF: 0.03496
    Word: 提问, TF-IDF: 0.03496
    Word: 总, TF-IDF: 0.03053
    Word: 混合战争, TF-IDF: 0.02738
    Word: 中国电子信息产业集团, TF-IDF: 0.02738
    Word: 执行长, TF-IDF: 0.02738
    Word: 干预, TF-IDF: 0.02738


Top words in document 3
    Word: 军力, TF-IDF: 0.04525
    Word: 黎智英, TF-IDF: 0.02657
    Word: 罗宾森, TF-IDF: 0.02586
    Word: 部署, TF-IDF: 0.02442
    Word: 周鹏, TF-IDF: 0.0232
    Word: 深红, TF-IDF: 0.01939
    Word: 痛批, TF-IDF: 0.01939
    Word: 苹果新闻网, TF-IDF: 0.01939
    Word: 台独, TF-IDF: 0.01856
    Word: 仿真, TF-IDF: 0.01856


Top wor

    Word: 

 , TF-IDF: 0.02319
    Word: 台, TF-IDF: 0.01742
    Word: 主持人, TF-IDF: 0.01739
    Word: 资料库, TF-IDF: 0.01356
    Word: 资料, TF-IDF: 0.01094
    Word: 台湾, TF-IDF: 0.01048
    Word: 怎幺, TF-IDF: 0.00832
    Word: 立委, TF-IDF: 0.00794
    Word: 人士, TF-IDF: 0.00769
    Word: 提到, TF-IDF: 0.0072




In [73]:
# 標註NER create dataframe
df_tfidf = pd.DataFrame(all_sorted)
ner = []
for i in range(len(final_text)):
    for k in range(len(df_tfidf)):
        if df_tfidf['doc'][k] == i+1:
            tempDF = df_all_pair[df_all_pair['cat'] ==i+1].drop_duplicates()
            tempDF.reset_index(inplace = True)
            for s in range(len(tempDF)):
                if df_tfidf['words'][k] ==  tempDF['words'][s]:
                    ner.append(tempDF['ner'][s])
                    break
df_tfidf['ner'] = ner

In [84]:
df_tfidf[df_tfidf['ner']=='r']

,words,scores,doc,ner
86,那个人,0.010006,10,r
225,那个人,0.010017,25,r
403,这场,0.016404,45,r
426,怎幺,0.008321,49,r


#### 計算詞向量

In [88]:
f = open('/Users/csti-user/Downloads/sgns.merge.word','r')
k = f.readlines()
f.close()

In [89]:
for i in k:
    print(i)

1292607 300

， 0.102387 0.111146 0.263319 0.023152 -0.016981 0.024967 -0.019429 0.148500 0.044953 0.231089 -0.195045 -0.060415 0.229814 0.125847 0.174458 0.186934 0.047486 0.122592 -0.139524 0.219931 0.078783 0.144782 -0.160593 -0.081857 0.083243 0.058926 0.177047 -0.171374 0.018400 -0.027682 -0.004983 -0.076983 0.131640 -0.113488 0.167113 -0.121429 0.063881 0.171087 -0.145927 0.083276 -0.170163 -0.074225 0.103492 0.093889 -0.155911 0.163988 -0.182002 0.157361 -0.191924 0.089651 0.068731 0.142282 -0.113051 0.132194 -0.326704 0.131645 -0.046067 -0.199591 -0.263973 -0.104769 -0.048914 0.014195 -0.019854 0.056857 0.059569 0.091732 0.083272 -0.167209 -0.033752 0.211743 0.035805 -0.000383 -0.089494 -0.012996 0.137302 -0.261788 0.087270 -0.167092 -0.279155 0.047554 -0.159121 -0.149205 0.235909 -0.279607 0.054076 -0.081279 0.121545 0.167018 -0.118002 0.371543 -0.148008 0.089215 0.042067 0.046976 -0.012934 0.140683 0.182944 -0.073797 0.193460 -0.133254 0.026854 -0.080276 0.144142 -0.304156 0.0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [175]:
from gensim.models.word2vec import Word2Vec

In [176]:
w2v = pd.DataFrame(columns=['texts', 'idx'])
for idx, arr in enumerate(all_pair):
#     arr = arr.tolist()
    w2v = w2v.append({'texts':arr, 'idx': idx}, ignore_index=True)
w2v

,texts,idx
0,"[深圳, 信息技术限公司, 企查查提示注册成功请交易理优惠券处查看天体验券查下, 浏览量, ...",0
1,"[网站, 已, 下线, 王雪峰, 鼓吹, 混合战争, 国际, 网站, 已, 下线, 王雪峰,...",1
2,"[解密, 華, 數據, 立場, 深紅, 痛批, 黎智英, 與, 台獨, 工程師, 曾, 研發...",2
3,"[川普, 长女, 例, 演示, 搜集, 开源, 信息深圳, 伊万卡大纪元澳大利亚新闻网×搜寻...",3
4,"[華, 數據, 川普, 長女, 為, 例, 演示, 蒐集, 開源, 信息, 深圳, 華, 數...",4
5,"[深圳華, 數據, 信息, 技術, 維基百科, 百科, 全, 書, 深圳華, 數據, 信息,...",5
6,"[深圳, 華, 數據, 看, 中國, 國企, 監控, 資料庫, 外洩, 消息, 中文, 分類...",6
7,[©],7
8,"[收集, 美国, 大, 目标, 名人, 军方, 信息, 搜尋, 關鍵字, 收集, 美国, 大...",8
9,"[中国, 大规模, 监控, 泄露, 包括, 万, 澳洲人, 人, 信息, 中国, 大规模, ...",9


In [177]:
for idx, i in enumerate(w2v.texts):
    if len(i) == 0:
        w2v = w2v.drop([idx])
w2v

,texts,idx
0,"[深圳, 信息技术限公司, 企查查提示注册成功请交易理优惠券处查看天体验券查下, 浏览量, ...",0
1,"[网站, 已, 下线, 王雪峰, 鼓吹, 混合战争, 国际, 网站, 已, 下线, 王雪峰,...",1
2,"[解密, 華, 數據, 立場, 深紅, 痛批, 黎智英, 與, 台獨, 工程師, 曾, 研發...",2
3,"[川普, 长女, 例, 演示, 搜集, 开源, 信息深圳, 伊万卡大纪元澳大利亚新闻网×搜寻...",3
4,"[華, 數據, 川普, 長女, 為, 例, 演示, 蒐集, 開源, 信息, 深圳, 華, 數...",4
5,"[深圳華, 數據, 信息, 技術, 維基百科, 百科, 全, 書, 深圳華, 數據, 信息,...",5
6,"[深圳, 華, 數據, 看, 中國, 國企, 監控, 資料庫, 外洩, 消息, 中文, 分類...",6
7,[©],7
8,"[收集, 美国, 大, 目标, 名人, 军方, 信息, 搜尋, 關鍵字, 收集, 美国, 大...",8
9,"[中国, 大规模, 监控, 泄露, 包括, 万, 澳洲人, 人, 信息, 中国, 大规模, ...",9


In [178]:
corpus = w2v.texts.sample(frac=1)
model = Word2Vec(corpus, window=3) #陽春型 - 總共1602個詞，每個詞有100維

2021-05-21 14:01:53,483 - INFO - collecting all words and their counts
2021-05-21 14:01:53,484 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-05-21 14:01:53,492 - INFO - collected 6079 word types from a corpus of 30981 raw words and 48 sentences
2021-05-21 14:01:53,493 - INFO - Loading a fresh vocabulary
2021-05-21 14:01:53,498 - INFO - effective_min_count=5 retains 1185 unique words (19% of original 6079, drops 4894)
2021-05-21 14:01:53,498 - INFO - effective_min_count=5 leaves 23112 word corpus (74% of original 30981, drops 7869)
2021-05-21 14:01:53,503 - INFO - deleting the raw counts dictionary of 6079 items
2021-05-21 14:01:53,503 - INFO - sample=0.001 downsamples 63 most-common words
2021-05-21 14:01:53,504 - INFO - downsampling leaves estimated 20147 word corpus (87.2% of prior 23112)
2021-05-21 14:01:53,506 - INFO - estimated required memory for 1185 words and 100 dimensions: 1540500 bytes
2021-05-21 14:01:53,507 - INFO - resetting layer weights

In [179]:
## 檢驗訓練好的詞向量模型
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [181]:
most_similar(model,['王雪峰','周鹏'])

,王雪峰,cos,周鹏,cos
0,公司,0.999887,通知,0.996905
1,資料庫,0.999879,时,0.996707
2,深圳,0.999878,理,0.996673
3,中國,0.999871,国,0.996654
4,與,0.999861,包括,0.996611
5,人,0.999860,版权,0.996610
6,年,0.999857,名單,0.996585
7,已,0.999856,並,0.996583
8,數據,0.999852,英国,0.996573
9,們,0.999851,蒐集,0.996561


In [183]:
model.wv.most_similar('周鹏')

[('通知', 0.9969050884246826),
 ('时', 0.9967068433761597),
 ('理', 0.9966732263565063),
 ('国', 0.9966539740562439),
 ('包括', 0.9966112971305847),
 ('版权', 0.9966099858283997),
 ('名單', 0.9965848326683044),
 ('並', 0.9965826272964478),
 ('英国', 0.9965730905532837),
 ('蒐集', 0.9965614676475525)]

In [184]:
model.wv.vocab.keys()  #model.wv.vectors  #

dict_keys(['字', '面包屑', '中国', '大', '德国', '声', '介绍', '提供', '服务', '信息', '保护', '声明', '互联网', '处', '留下', '种', '人', '观点', '商业', '公司', '早', '利', '些', '广告', '现', '发现', '正', '海量', '监控', '军事', '指', '网路', '上', '根', '研究', '调查', '系统', '搜集', '作', '情报', '中国政府', '单位', '创办人', '波特', '独立', '研究员', '鲍丁', '包括', '盛顿邮报', '内', '家', '新闻', '机构', '份', '部份', '基础', '库', '下载', '重建', '约', '完整', '估计', '文', '说', '民主', '国家', '法律', '外', '外国', '带', '真实', '威胁', '美国', '澳大利亚', '英国', '日电讯报', '卫报', '媒体', '周', '月日', '步', '公开', '报导', '内容', '未', '受', '才', '名', '深圳', '科技', '年', '收集', '社交', '政府', '客户', '海外', '关键', '称', '关', '政治', '人物', '国', '部署', '细节', '舆', '分析', '营销', '文件', '包含', '万人', '中', '报告', '指出', '泄露', '恢复', '记录', '万名', '美国人', '近', '英国人', '首相', '约翰逊', '莫里森', '成员', '名人', '担', '重', '海军', '军官', '推文', '档案', '家庭', '片', '直', '人士', '都', '然', '没', '证', '显示', '招聘', '描述', '目标', '军队', '回应', '采访', '时', '相关', '报道', '位', '姓', '业务', '负责人', '不', '整合', '模式', '合作', '伙伴', '特', '存', '样', '平台', '军方', '集团', '隐私', '倡议', '长期', '关注', '非', '大规模', '开源'

In [96]:
model.wv.vectors

array([[ 0.51522   , -0.34755117,  0.02856257, ..., -0.5155937 ,
        -0.11341139,  0.17871518],
       [ 0.4993312 , -0.3464844 ,  0.03208831, ..., -0.49490416,
        -0.11454304,  0.17727539],
       [ 0.54264355, -0.33415547,  0.13614203, ..., -0.51091266,
        -0.10583877,  0.1650165 ],
       ...,
       [ 0.0592349 , -0.04136999,  0.00962222, ..., -0.0562446 ,
        -0.01518707,  0.01744464],
       [ 0.04802441, -0.02979744,  0.00921943, ..., -0.04377791,
        -0.00779128,  0.01104703],
       [ 0.05102098, -0.03945457,  0.00423029, ..., -0.05812755,
        -0.00960899,  0.01804923]], dtype=float32)

In [12]:
corpus = [' '.join(x) for x in doc_terms]
corpus[2]

'振华 黎智英 壹会员 优惠 娱乐 周鹏 演示 相关 合作 服务 王雪峰 壹传媒黎智英 倾向偏 深红 中国政府 国安文武小英 林锡耀 洪耀福 深圳振华数据信息技术有限公司 澳洲网络安全公司罗宾森 David Robinson振华 删除 腾讯 所在 振华公司 采集 分析 直接 展示 罗宾森 拓尔思信息技术股份有限公司 中共中央办公厅 国务院 新华社 央视振华 殷琪 罗文嘉 胡亦嘉 研发 天津大学 深圳振华数据信息技术有限公司周鹏 研究 领英 大量 张锡铭 多 评估 近 搜集 作战 操控 认知 岑子杰 祸港乱港 林飞帆 澳洲广播公司 IBM王雪峰 中国共产党振华 成功 关连 报导 严重 调查 冗长 警戒 陈时中 出版 万华 屏东 全面 侯友宜 东山国三生 苗栗零 谷关 移动 聚扩 壹传媒 苹果日报 分级 APPLE ONLINE All rights reserved.'

In [197]:
OTP = df_tfidf[df_tfidf['ner']=='ORG+TITLE+PER']
OP = df_tfidf[df_tfidf['ner']=='ORG+PER']
mergeDF = pd.concat([OP,OTP])
mergeDF.reset_index(drop = True, inplace = True)
mergeDF

,words,scores,doc,ner
0,国际商业机器公司王雪峰,0.353802,2,ORG+PER
1,悉尼科技大学王雪峰,0.353802,2,ORG+PER
2,公司王雪峰,0.164762,2,ORG+PER
3,壹传媒黎智英,0.114465,3,ORG+PER
4,澳洲网络公司罗宾森,0.114465,3,ORG+PER
...,...,...,...,...
122,中国国家安总统安倍晋三,0.090507,38,ORG+TITLE+PER
123,中共总统川普,0.090507,38,ORG+TITLE+PER
124,中共部长吴钊燮,0.090507,38,ORG+TITLE+PER
125,北京大学教授鲍尔丁,0.116670,39,ORG+TITLE+PER


In [198]:
Counter(mergeDF.doc).most_common()

[(7, 5),
 (15, 5),
 (19, 5),
 (20, 5),
 (24, 5),
 (38, 5),
 (4, 4),
 (13, 4),
 (14, 4),
 (16, 4),
 (17, 4),
 (26, 4),
 (32, 4),
 (33, 4),
 (34, 4),
 (36, 4),
 (39, 4),
 (40, 4),
 (2, 3),
 (3, 3),
 (5, 3),
 (9, 3),
 (10, 3),
 (12, 3),
 (21, 3),
 (23, 3),
 (27, 3),
 (28, 3),
 (44, 3),
 (45, 3),
 (6, 2),
 (22, 2),
 (31, 2),
 (11, 1),
 (25, 1),
 (29, 1),
 (37, 1),
 (43, 1),
 (47, 1),
 (49, 1)]

### K-means文字聚類，看可否分出幾個clusters

In [173]:
# df = pd.DataFrame(columns=['data','index'])
# lac1 = LAC(mode='seg')  
# lac1.load_customization('custom_dict.txt')  # 增加title字典
# eachTexts = []
index = []
# for idx, texts in enumerate(final_text):
#     texts = cc.convert(texts) # 繁轉簡
#     texts = multi_repl(texts,trim_dict)  # 刪除停用詞
#     texts = re.sub(r'[a-zA-Z]',"", texts)  # 去除英文
#     texts = lac1.run(texts)  # 進行分詞
#     df = df.append({'data':texts, 'index': idx+1},ignore_index=True)
#     index.append(idx)
for idx, t in enumerate(df.data):
    for a in t:
        eachTexts.append(a)
        index.append(str(idx))

In [154]:
class KMeans:
    def cal_dist(self, p0, p1):
        """
        比較兩點的距離
        """
        return np.sqrt(np.sum((p0-p1)**2))
    
    def nearest_cluster_center(self, point, cluster_centers):
        """
        找到距離 point 最近的中心點
        """
        min_dist = float("inf")
        m = cluster_centers.shape[0]
        for i in range(m):
            d = self.cal_dist(point, cluster_centers[i])
            if min_dist > d:
                min_dist = d
        return min_dist 

    def get_centroids(self, datapoints, k):
        """
        K-means++ 演算法，取得初始化中心點
        """
        clusters = np.array([random.choice(datapoints)])
        dist = np.zeros(len(datapoints))
        
        for i in range(k-1):
            sum_dist = 0
            for j, point in enumerate(datapoints):
                dist[j] = self.nearest_cluster_center(point, clusters)
                sum_dist += dist[j]
            
            sum_dist *= random.random()
            for j, d in enumerate(dist):
                sum_dist = sum_dist - d
                if sum_dist <= 0:
                    clusters = np.append(clusters, [datapoints[j]], axis=0)
                    break
        
        return clusters
        
        
    def kmeans_plus_plus(self, datapoints, k=2):
        """
        K-means 演算法
        """
        # 定義資料維度
        d = datapoints.shape[1]
        # 最大的迭代次數
        Max_Iterations = 1000

        cluster = np.zeros(datapoints.shape[0])
        prev_cluster = np.ones(datapoints.shape[0])

        cluster_centers = self.get_centroids(datapoints, k)

        iteration = 0
        while np.array_equal(cluster, prev_cluster) is False or iteration > Max_Iterations:
            iteration += 1
            prev_cluster = cluster.copy()

            # 將每一個點做分群
            for idx, point in enumerate(datapoints):
                min_dist = float("inf")
                for c, cluster_center in enumerate(cluster_centers):
                    dist = self.cal_dist(point, cluster_center)
                    if dist < min_dist:
                        min_dist = dist  
                        cluster[idx] = c   # 指定該點屬於哪個分群

            # 更新分群的中心
            for k in range(len(cluster_centers)):
                new_center = np.zeros(d)
                members = 0
                for point, c in zip(datapoints, cluster):
                    if c == k:
                        new_center += point
                        members += 1
                if members > 0:
                    new_center = new_center / members
                cluster_centers[k] = new_center

        return cluster

In [145]:
# 使用 tf-idf 向量化
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(eachTexts)
tfidf = tfidf.toarray()
tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [174]:
k = 4
Kmeans_cluster = KMeans()
speech_cluster_result = Kmeans_cluster.kmeans_plus_plus(tfidf, k)
cluster = [[] for _ in range(k)]

for idx, c in enumerate(speech_cluster_result):
    cluster[int(c)].append(index[idx])
    
for c, result in enumerate(cluster):
    print('Cluster {}: {}'.format(c, ' '.join(result)))

Cluster 0: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

## Single Doc Information Extraction

In [ ]:
'''
研究人員選擇某一篇文章，直接回傳該篇「網址」
直接進行爬蟲、分詞、命名實體標注
保留某些特定的： (Who)人物 組織 職位、(What)事件
一樣要做組合NER
'''

In [92]:
def getFinalText(google_search_rs):
    html_text = []
    for idx, link in enumerate(google_search_rs):
        resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
        soup = BeautifulSoup(resp.content, 'html.parser')
        a_tag = soup.find_all('a')
        for a in a_tag:
            a.decompose()   
        html_text.append(soup.get_text()) 
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 去除英文
    final_text = [re.sub(r'[a-zA-Z]',"", texts) for texts in temp]
    return final_text

In [281]:
def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

In [108]:
def stopwords():
    # Load stop words
    f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
    k = f.readlines()
    f.close()
    stopwords = [lines.strip('\n') for lines in k]

    # 建構stopwords
    trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim = trim + stopwords  # list of all stopwords
    return trim

In [283]:
def ner_extract(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    ner_model = lac.run(final_text)

    ner_pair = []
    trim = stopwords()
    docs = pd.DataFrame(columns=['texts', 'doc'])  # dataframe 每篇文章為一列
    for doc, (word, ner) in enumerate(ner_model):
        temp = []
        for num in range(len(word)):
            if word[num] not in trim: # remove stop words
                word[num] = multi_repl(word[num])
                if word[num] != '' and ner[num] != '' and ner[num] != 'f' and ner[num] != 'nw' and ner[num] != 'm':
                    temp.append(word[num])
                    ner_pair.append((word[num],ner[num],doc+1))
        docs = docs.append({'texts': temp, 'doc': doc+1}, ignore_index = True)
    ner_combine(ner_pair)
    return ner_pair

In [101]:
def ner_combine(ner_pair):
    for idx, (word, ner, cat) in enumerate(ner_pair):
        if (idx + 1) == len(ner_pair) or (idx + 2) == len(ner_pair):
            break
        else:  # 直接將組合詞加入原先分詞好的list (ner_pair)
            n1_ner = ner_pair[idx+1]
            if cat == n1_ner[2]:
                n2_ner = ner_pair[idx+2]
                if ner == 'ORG' and n1_ner[1] == 'PER':
                    new_wd = (word+n1_ner[0],'ORG+PER',cat)
                    del ner_pair[idx:idx+2]
                    ner_pair.insert(idx,new_wd) 
                elif ner == 'ORG' and n1_ner[1] == 'TITLE' and n2_ner[1] == 'PER':
                    new_wd = (word+n1_ner[0]+n2_ner[0],'ORG+TITLE+PER',cat)
                    del ner_pair[idx:idx+3]
                    ner_pair.insert(idx,new_wd)

In [288]:
def auto_extract(pair):
    df = pd.DataFrame(pair, columns=['word','ner','doc'])
    ## dataframe including some ner: org,title, per
    keywords = pd.concat((df[df['ner'] == 'ORG+TITLE+PER'],df[df['ner'] == 'ORG+PER'], df[df['ner'] == 'ORG'], 
                          df[df['ner'] == 'PER'])).reset_index()
    keywords = keywords.drop(columns=['index'])
    
    ## 15 most common words dataframe
    cnt = pd.DataFrame(Counter(keywords.word).most_common(15),columns=['words', 'cnt'])
    ## add ner
    cnt['ner'] = [keywords.ner[k] for i in range(len(cnt)) for k in range(len(keywords.word.unique())) if cnt.words[i] == keywords.word.unique()[k]]
    cnt = cnt.drop(columns=['cnt'])
    return cnt

In [402]:
def manual_input(pair):
    df = pd.DataFrame(pair, columns=['word','ner','doc'])
    ## dataframe including some ner: org, title, per
    keywords = pd.concat((df[df['ner'] == 'ORG+TITLE+PER'],df[df['ner'] == 'ORG+PER'], df[df['ner'] == 'ORG'], 
                          df[df['ner'] == 'PER'], df[df['ner'] == 'TITLE'])).sort_index()
    keywords = keywords.reset_index()
    keywords = keywords.drop(columns =['index'])
    
    cnt = auto_extract(pair)
    cntList = cnt.words.unique().tolist()
    
    idx = [i for i in range(len(keywords)) if keywords.word.iloc[i] in cntList]
    keywords = keywords.drop(idx).reset_index()
    keywords = keywords.drop(columns=['index', 'doc'])
    return keywords

In [399]:
## Automatic Word Segmentation and Name Entity Recognition
url = ['http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html']
final_text = getFinalText(url)
pair = ner_extract(final_text)  # [('北京科学技术情报学会', 'ORG', 1),....

## programme: NER extract from single document, automatically show 10 to 15 keywords
auto_extract(pair) # return 15 keywords created automatically

## manual input: generate word segments from single document
manual_input(pair)

,words,ner
0,北京科学技术情报学会,ORG
1,北京市科学技术协会,ORG
2,中国科协,ORG
3,吴晨生,ORG
4,北京市科学技术协会学会部副部长李斌,ORG+TITLE+PER
5,北京科学技术情报学会理事长谢威,ORG+TITLE+PER
6,科技部中国科学技术信息研究所所长戴国强,ORG+TITLE+PER
7,科技部中国科技发展战略研究院调查统计中心主任邓大胜,ORG+TITLE+PER
8,中国电信研究院技术总监钱兵,ORG+TITLE+PER
9,中国国防科技信息中心总工真溱,ORG+TITLE+PER
